<a href="https://colab.research.google.com/github/gdm63/class-01-dev-env/blob/main/FineTuningRevisedINFO4940.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


# New Section

In [ ]:
!pip install -q datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import Dataset
import torch
import pandas as pd

# Check GPU availability and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
model_name = "EleutherAI/pythia-410m"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# model = AutoModelForCausalLM.from_pretrained(model_name)

# Move model to device
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
def prepare_dataset(questions, answers):
    return Dataset.from_dict({"text": [f"Arguement: {q}\nVerdict: {a}" for q, a in zip(questions, answers)]})

def tokenize_function(examples, tokenizer, max_length):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_length)

questions_df = pd.read_csv("/content/complete_antitrust_cases_summary_100.csv")

dataset = prepare_dataset(questions_df["Plaintiff Argument"], questions_df["Outcome"])

In [ ]:
dataset

Dataset({
    features: ['text'],
    num_rows: 100
})

In [ ]:
# Set up data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Preprocess the dataset
max_length = 64
tokenized_dataset = dataset.map(
    lambda examples: tokenize_function(examples, tokenizer, max_length),
    batched=True,
    remove_columns=dataset.column_names
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
train_test = tokenized_dataset.train_test_split(test_size=0.1)

In [ ]:
train_test

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 90
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 10
    })
})

In [ ]:
dataset[0], tokenized_dataset[0]

({'text': "Arguement: DOJ alleged Microsoft unlawfully maintained its OS monopoly by bundling Internet Explorer with Windows, preventing OEMs from promoting Netscape. Internal memos indicated Microsoft's intent to 'cut off Netscape's air supply' by using exclusive deals with manufacturers and ISPs.\nVerdict: Initially ordered to break up, Microsoft reached a settlement requiring it to share APIs and allow third-party software integration."},
 {'input_ids': [3163,
   5162,
   972,
   25,
   27055,
   4260,
   5413,
   48843,
   9456,
   663,
   7294,
   22036,
   416,
   12207,
   1359,
   4455,
   19142,
   351,
   3964,
   11,
   12174,
   29671,
   82,
   422,
   11560,
   27811,
   36435,
   13,
   18628,
   44940,
   8203,
   5413,
   338,
   6824,
   284,
   705,
   8968,
   572,
   27811,
   36435,
   338,
   1633,
   5127,
   6,
   416,
   1262,
   8568,
   7529,
   351,
   11372,
   290,
   28677,
   13,
   198,
   13414,
   11600,
   25,
   30900,
   6149,
   284,
   2270,
   

In [ ]:
output_dir = "./fine_tuned_causal_model"

# Set up training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    report_to="none",
    per_device_train_batch_size=8,
    num_train_epochs=33,
    save_steps=1000,
    save_total_limit=2,
    logging_dir=f"{output_dir}/logs",
    logging_steps=10,
    learning_rate=2e-5,
    warmup_steps=5,
    weight_decay=0.01,
    evaluation_strategy="steps",
    eval_steps=250, #new
    load_best_model_at_end=True, #new
  )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_test["train"],
    eval_dataset=train_test["test"],
    data_collator=data_collator,
)
trainer.can_return_loss = True

# Start training
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 9.06 MiB is free. Process 14921 has 14.74 GiB memory in use. Of the allocated memory 13.68 GiB is allocated by PyTorch, and 946.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.evaluate()

In [ ]:
def generate(s, new_tokens=50):
    input = tokenizer(s, return_tensors="pt").to(device)
    output = model.generate(**input, max_new_tokens=new_tokens, do_sample=True)
    print(tokenizer.batch_decode(output)[0])

In [ ]:
generate("Argument: DOJ claimed AT&T’s monopoly on long-distance services resulted from control over local Bell companies, preventing competitors from accessing the network.\nVerdict: ")

In [ ]:
generate("Argument: DOJ argued that American Express’ anti-steering rules prevented merchants from promoting alternative cards with lower fees, limiting competition.\nVerdict: ")

In [ ]:
generate("Argument: EU claimed TikTok engaged in unfair data collection practices impacting user privacy and limiting competition for user attention.\nVerdict: ")

In [ ]:
generate("Argument: EU argued Meta leveraged its dominance in social networking to promote its Marketplace, limiting rival online marketplaces.\nVerdict: ")

In [ ]:
generate("Argument: EU alleged Amazon used third-party seller data to undermine competition.\nVerdict: ")

In [ ]:
generate("Argument: On Jan. 24, 2023, the DOJ filed a lawsuit against Google accusing the company of having monopolized the ad tech market. According to the department's antitrust lawsuit, Google capitalized on its combination of ad tech tools — particularly its ad exchange and publisher ad server — to corner the programmatic ad market.\nVerdict: ")

In [ ]:
generate("Argument: On September 24, 2024, the DOJ filed a civil antitrust lawsuit today against Visa for monopolization and other unlawful conduct in debit network markets in violation of Sections 1 and 2 of the Sherman Act. The complaint alleges that Visa illegally maintains a monopoly over debit network markets by using its dominance to thwart the growth of its existing competitors and prevent others from developing new and innovative alternatives./nVerdict: ")

In [ ]:
generate("Argument: Margrethe Vestager, the European Commissioner for Competition, has accused Apple of failing to comply with the Digital Markets Act (DMA) over its app store rules and charges. The EU alleges that Apple designed its new business model to discourage app developers and end users from taking advantage of the opportunities afforded to them by the DMA./nVerdict: ")

In [ ]:
generate("Argument: The DOJ sued to break up Ticketmaster and its parent company Live Nation alleging that they have illegally monopolized the live music industry harming fans, artists and venues, through a mix of exclusive contracts, self-preferencing, retaliation and acquisitions./nVerdict: ")

In [ ]:
generate("Argument: The Biden administration sued to stop the $25 billion deal to merge supermarket giants Kroger and Albertsons in February, arguing that it would lead to higher food costs and fewer and worse jobs for unionized workers. /nVerdict: ")